In [268]:
import os

import numpy as np
import pandas as pd


import matplotlib.pyplot as plt

import sklearn.model_selection

import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.predictor import csv_serializer

In [269]:
session = sagemaker.Session()  #Sagemaker Necessities, can be ignored
role = get_execution_role()    #Sagemaker Necessities, can be ignored

In [305]:
df=pd.read_csv('parkinsons.data')      # Reading the dataset and dropping a column that serves no purpose
df= df.drop(['name'], axis=1)
df.head(100)


,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,157.447,163.267,149.605,0.00369,0.00002,0.00201,0.00197,0.00602,0.03272,0.283,...,0.05439,0.01018,21.693,1,0.447285,0.705658,-6.247076,0.180528,2.344348,0.164916
96,159.116,168.913,144.811,0.00342,0.00002,0.00178,0.00184,0.00535,0.03381,0.307,...,0.05417,0.00852,22.663,1,0.366329,0.693429,-6.417440,0.194627,2.473239,0.151709
97,125.036,143.946,116.187,0.01280,0.00010,0.00743,0.00623,0.02228,0.03886,0.342,...,0.06406,0.08151,15.338,1,0.629574,0.714485,-4.020042,0.265315,2.671825,0.340623
98,125.791,140.557,96.206,0.01378,0.00011,0.00826,0.00655,0.02478,0.04689,0.422,...,0.07625,0.10323,15.433,1,0.571010,0.690892,-5.159169,0.202146,2.441612,0.260375


In [306]:
df.columns.values  # Just getting an idea of the column titles we have

array(['MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP',
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA',
       'spread1', 'spread2', 'D2', 'PPE'], dtype=object)

In [307]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

dfs=pd.DataFrame(scaler.fit_transform(df.astype(float)))#Applying standardization to our dataset
dfs.columns=df.columns.values   #Making sure we add back the previous column names

labels=dfs.loc[:,'status'].values    #Giving the object 'labels' the classification of each sample ( 1 or 0)
data= dfs.drop(['status'], axis=1)   #retreiving the dataset, dropping the Y-value, to only have features


X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(data, labels, test_size=0.33)
X_train, X_val, Y_train, Y_val = sklearn.model_selection.train_test_split(X_train, Y_train, test_size=0.33)

In [332]:
X_test  # Just have an idea of what we are working with 

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,MDVP:APQ,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
43,0.891129,0.299436,0.961501,0.035896,0.011858,0.042871,0.043408,0.042703,0.073580,0.056697,...,0.040738,0.106111,0.019416,0.597578,0.469298,0.238978,0.211797,0.342156,0.370757,0.150903
187,0.163059,0.978074,0.165636,0.031449,0.051383,0.022640,0.030011,0.022636,0.031587,0.026294,...,0.032545,0.033894,0.019575,0.673901,0.634454,0.356972,0.290360,0.248441,0.324217,0.195368
48,0.197093,0.054025,0.289526,0.113088,0.130435,0.048651,0.059486,0.048643,0.060161,0.047658,...,0.054675,0.077160,0.024636,0.598269,0.753713,0.634953,0.275827,0.585563,0.292115,0.185010
132,0.178859,0.047089,0.122739,0.056544,0.090909,0.048651,0.041801,0.048804,0.022549,0.017256,...,0.015009,0.034279,0.026673,0.607494,0.500260,0.585747,0.457218,0.559713,0.486253,0.388689
157,0.172496,0.065452,0.203122,0.522554,0.565217,0.505299,0.335477,0.505218,0.361329,0.290058,...,0.168236,0.415073,0.340039,0.432171,0.872594,0.288854,0.817311,0.668454,0.685663,0.448951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,0.137863,0.992831,0.239202,0.115947,0.169960,0.096339,0.127010,0.096484,0.136389,0.115859,...,0.094724,0.176916,0.017761,0.639559,0.589287,0.857549,0.465716,0.676157,0.224845,0.374713
164,0.081154,0.083050,0.117598,0.234752,0.328063,0.204721,0.197749,0.204848,0.169801,0.153657,...,0.103683,0.216459,0.121495,0.406649,0.967864,0.399512,0.910441,0.879008,0.505873,0.668342
78,0.041054,0.000000,0.142711,0.139771,0.209486,0.126686,0.128617,0.126826,0.205222,0.146261,...,0.132935,0.259597,0.031734,0.545964,0.906209,0.818028,0.515241,0.547523,0.264458,0.446279
76,0.142881,0.095290,0.238955,0.105464,0.130435,0.097303,0.105573,0.097287,0.277981,0.217749,...,0.152002,0.377070,0.026737,0.606722,0.543580,0.859322,0.395776,0.753896,0.260085,0.309383


In [308]:
data_dir = '../data/training_files'      #We are making sure the pathway we will add training,valid, and testing files exist
data_dir_test = '../data/training_files/test_folder'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [309]:
X_train= pd.DataFrame(X_train)
X_val= pd.DataFrame(X_val)
X_test= pd.DataFrame(X_test)
    #Seperating the dataset into X and Y variables for their specific type (train, validation, testing)
Y_train= pd.DataFrame(Y_train)
Y_val= pd.DataFrame(Y_val)
Y_test= pd.DataFrame(Y_test)


In [310]:
X_test.to_csv(os.path.join(data_dir_test, 'test.csv'), header=False, index=False)

#Here we prepare our data to the files we specified, in S3, this is sagemaker specific, it can be ignored

pd.concat([Y_train, X_train], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)
pd.concat([Y_val, X_val], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)

In [311]:
prefix = 'training_files-xgboost-tuning-HL'

#Here we upload our data to the files we specified, in S3, this is sagemaker specific, it can be ignored


test_location = session.upload_data(os.path.join(data_dir_test, 'test.csv'), key_prefix=prefix)
val_location = session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

In [312]:
# we use this utility method to construct the image name for the training container. This can be ignored
container = get_image_uri(session.boto_region_name, 'xgboost')

# Now that we know which container to use, we can construct the estimator object.Can be ignored

xgb = sagemaker.estimator.Estimator(container, # The name of the training container
                                    role,      # The IAM role to use (our current role in this case)
                                    train_instance_count=1, # The number of instances to use for training
                                    train_instance_type='ml.m4.xlarge', # The type of instance ot use for training
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                                                        # Where to save the output (the model artifacts)
                                    sagemaker_session=session) # The current SageMaker session

	get_image_uri(region, 'xgboost', '1.0-1').


In [313]:
#Here we have our hyperparameters, which are extremely similar to the solution code that got 94% accuracy

xgb.set_hyperparameters(max_depth=3,
                        eta=0.2,
                        gamma=0,
                        min_child_weight=1,
                        subsample=1,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        num_round=100)

In [314]:
#This is the location of our training and validation set
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')

#Here we fit our model to the training and validation sets
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

2020-07-16 06:35:18 Starting - Starting the training job...
2020-07-16 06:35:20 Starting - Launching requested ML instances......
2020-07-16 06:36:23 Starting - Preparing the instances for training...
2020-07-16 06:37:15 Downloading - Downloading input data...
2020-07-16 06:37:32 Training - Downloading the training image..Arguments: train
[2020-07-16:06:37:53:INFO] Running standalone xgboost training.
[2020-07-16:06:37:53:INFO] File size need to be processed in the node: 0.06mb. Available memory size in the node: 8494.63mb
[2020-07-16:06:37:53:INFO] Determined delimiter of CSV input is ','
[06:37:53] S3DistributionType set as FullyReplicated
[06:37:53] 136x22 matrix with 2943 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2020-07-16:06:37:53:INFO] Determined delimiter of CSV input is ','
[06:37:53] S3DistributionType set as FullyReplicated
[06:37:53] 77x22 matrix with 1660 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&d

In [315]:
#This is sagemaker specific, we are just specifying the training job name for our model
xgb_attached = sagemaker.estimator.Estimator.attach('xgboost-2020-07-16-06-35-18-680')

2020-07-16 06:38:04 Starting - Preparing the instances for training
2020-07-16 06:38:04 Downloading - Downloading input data
2020-07-16 06:38:04 Training - Training image download completed. Training in progress.
2020-07-16 06:38:04 Uploading - Uploading generated training model
2020-07-16 06:38:04 Completed - Training job completedArguments: train
[2020-07-16:06:37:53:INFO] Running standalone xgboost training.
[2020-07-16:06:37:53:INFO] File size need to be processed in the node: 0.06mb. Available memory size in the node: 8494.63mb
[2020-07-16:06:37:53:INFO] Determined delimiter of CSV input is ','
[06:37:53] S3DistributionType set as FullyReplicated
[06:37:53] 136x22 matrix with 2943 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2020-07-16:06:37:53:INFO] Determined delimiter of CSV input is ','
[06:37:53] S3DistributionType set as FullyReplicated
[06:37:53] 77x22 matrix with 1660 entries loaded from /opt/ml/input/data/validation?format=csv&label_

In [316]:
xgb_transformer = xgb_attached.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')
#Making it available for BatchTesting

In [317]:
xgb_transformer.transform(test_location, content_type='text/csv', split_type='Line')
#Now to actually test the model on the test data

In [318]:
xgb_transformer.wait()

....................Arguments: serve
[2020-07-16 06:43:32 +0000] [1] [INFO] Starting gunicorn 19.7.1
[2020-07-16 06:43:32 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2020-07-16 06:43:32 +0000] [1] [INFO] Using worker: gevent
[2020-07-16 06:43:32 +0000] [36] [INFO] Booting worker with pid: 36
[2020-07-16 06:43:32 +0000] [37] [INFO] Booting worker with pid: 37
[2020-07-16 06:43:32 +0000] [38] [INFO] Booting worker with pid: 38
[2020-07-16:06:43:32:INFO] Model loaded successfully for worker : 36
[2020-07-16 06:43:32 +0000] [39] [INFO] Booting worker with pid: 39
[2020-07-16:06:43:32:INFO] Model loaded successfully for worker : 37
[2020-07-16:06:43:32:INFO] Model loaded successfully for worker : 38
[2020-07-16:06:43:32:INFO] Model loaded successfully for worker : 39
[2020-07-16:06:43:59:INFO] Sniff delimiter as ','
[2020-07-16:06:43:59:INFO] Determined delimiter of CSV input is ','
[2020-07-16:06:43:59:INFO] Sniff delimiter as ','
[2020-07-16:06:43:59:INFO] Determined delimite

In [319]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir 
# This is just allows us to have access to the predictions file

download: s3://sagemaker-us-east-2-262034113260/xgboost-2020-07-16-06-35-18-680-2020-07-16-06-40-26-996/test.csv.out to ../data/training_files/test.csv.out


In [335]:
#This is the object that holds all of the predictions data
predictions = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)

#Here we will just convert that data to rounded integers so that they can ve classified as either 1 or 0
predictions = [round(num) for num in predictions.squeeze().values]

In [336]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test, predictions)

0.7384615384615385